In [39]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
import pandas as pd

In [40]:
model = pyo.AbstractModel()

In [41]:
room_population = pd.read_csv("SRO_input_grades.csv")

In [42]:
room_population

,Grade_ID,Population
0,PreKA,12
1,PreKB,14
2,PreKC,10
3,KindergartenA,16
4,KindergartenB,18
5,FirstA,15
6,FirstB,16
7,SecondA,32
8,ThirdA,16
9,ThirdB,17


In [43]:
room_population['Population'].sum()

np.int64(238)

In [44]:
model = pyo.AbstractModel()

In [45]:
model.GRADES = pyo.Set()
model.TIMESLOTS = pyo.Set()
model.ROOMS = pyo.Set()

In [46]:
model.assign = pyo.Var(model.GRADES, model.TIMESLOTS, model.ROOMS, domain = pyo.Binary)

In [47]:
model.POP = pyo.Param(model.GRADES)
model.CAP = pyo.Param(model.ROOMS)

In [48]:
@model.Objective(sense = pyo.maximize)
def profit(model):
    revenue = pyo.sum_product(model.assign)
    
    return revenue

In [49]:
@model.Constraint(model.TIMESLOTS, model.ROOMS)
def raw_material_constraint(model, t, r):
    return pyo.quicksum(model.assign[g, t, r] for g in model.GRADES) <= 1

In [75]:
@model.Constraint(model.GRADES, model.GRADES)
def equality(model, g, g_prime):
    if g == g_prime:
        return Constraint.Skip
    return (sum(model.assign[g, t, r] for t in model.TIMESLOTS for r in model.ROOMS) == 
                        sum(model.assign[g_prime, t, r] for t in model.TIMESLOTS for r in model.ROOMS))


'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>). This
is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [76]:
# @model.Constraint(model.GRADES)
# def equality_room_constraint(model, g):
#     return sum(model.assign[g, t, r] for t in model.TIMESLOTS for r in model.ROOMS) <= (len(model.TIMESLOTS) * len(model.ROOMS)) / len(model.GRADES) 

In [77]:
@model.Constraint(model.GRADES, model.TIMESLOTS)
def room(model, g, t):
    return pyo.quicksum(model.assign[g, t, r] for r in model.ROOMS) <= 1

'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.constraint.IndexedConstraint'>). This
is usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [78]:
@model.Constraint(model.GRADES, model.TIMESLOTS, model.ROOMS)
def room_capacity(model, g, t, r):
    return model.assign[g, t, r] * model.POP[g] <= model.CAP[r]

(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().


In [79]:
TIMESLOTS = ['8', '9', '10', '11', '12', '13', '14', '15']

In [80]:
input_rooms = pd.read_csv("SRO_input_rooms.csv")

In [81]:
input_rooms

,Room_ID,Capacity
0,A121,15
1,A120,13
2,A123,15
3,A137,12
4,A138,15
5,B142,12
6,B144,12
7,B146,12
8,B148,12
9,C153,12


In [82]:
instanceData = {None: {
    'GRADES': {None: room_population['Grade_ID']},
    'TIMESLOTS': {None: TIMESLOTS},
    'ROOMS': {None: input_rooms['Room_ID']},
    'POP': room_population.set_index(['Grade_ID']).to_dict()['Population'],
    'CAP': input_rooms.set_index(['Room_ID']).to_dict()['Capacity']
}}

In [83]:
instance = model.create_instance(instanceData)

ERROR: Rule failed when generating expression for Constraint equality with
index ('PreKA', 'PreKA'): NameError: name 'Constraint' is not defined
ERROR: Constructing component 'equality' from data=None failed:
        NameError: name 'Constraint' is not defined


NameError: name 'Constraint' is not defined

In [84]:
opt = SolverFactory('glpk')
sol = opt.solve(instance)

In [72]:
sol

{'Problem': [{'Name': 'unknown', 'Lower bound': 48.0, 'Upper bound': 48.0, 'Number of objectives': 1, 'Number of constraints': 2112, 'Number of variables': 1729, 'Number of nonzeros': 43200, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.4091939926147461}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [73]:
for g in instance.GRADES:
    val = 0
    for t in instance.TIMESLOTS:
        for r in instance.ROOMS:
            val += pyo.value(instance.assign[g, t, r])
    print(val)

4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
4.0
